# Modelo de clasificación con Perceptrón Multicapa 
## Autora: Sandra Alonso Paz, estudiante del Máster en Biología Computacional.

In [221]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

# Importación de los datos
### 1. Importar los datos del CSV

In [222]:
path ='dataset18.csv'
data = pd.read_csv(path)

### 2. Creamos las dos variables que utilizaremos en el modelo
   
    X = Atributos del modelo (columnas del CSV)
    
    Y = Columna target (la que queremos predecir a partir del resto de columnas)

In [223]:
X = data.iloc[:,2:23] #Seleccionamos todas las columnas quitando la columna Unnamed (id de la fila ) y la columna Target.
Y = []

# Como la columna target esta compuesta por valores categóricos ordinales (R y NR) convertimos en valores numéricos (R=0, NR=1) 
for i in range (len(data)):
    if data.Target[i]=='R': #R
        Y.append(0)
    else:                   #NR
        Y.append(1)  

# Creación del modelo de Perceptrón Multicapa
### 1. Separamos el conjunto de datos en entrenamiento y prueba

In [224]:
XTrain, XTest, yTrain, yTest = train_test_split(X, Y, test_size=0.15, random_state=125)

print('Tamaño del conjunto de entrenamiento:', len(XTrain))
print('Tamaño de la columna objetivo del conjunto de entrenamiento:', len(yTrain))
print('Tamaño del conjunto de prueba:', len(XTest))
print('Tamaño de la columna objetivo del conjunto de prueba:', len(yTest))

Tamaño del conjunto de entrenamiento: 45
Tamaño de la columna objetivo del conjunto de entrenamiento: 45
Tamaño del conjunto de prueba: 8
Tamaño de la columna objetivo del conjunto de prueba: 8


### 2. Instanciamos un GridSeacrgCV para averiguar que parámetros ajustan mejor el modelo

In [225]:
param_grid = {'max_iter': [200, 500, 1000],
            'activation': ['identity', 'logistic', 'tanh', 'relu'],
            'random_state': [125],
            'max_fun': [300, 500,1000, 5000, 10000, 15000, 20000],
            'hidden_layer_sizes': [3,5],
            'solver': ['lbfgs']}

# Creamos un GridSearchCV que permite evaluar y seleccionar de forma sistemática los parámetros de nuestro modelo. 
# Indicándole un modelo y los parámetros a probar, puede evaluar el rendimiento del primero en función de los 
# segundos mediante validación cruzada. 
clf = GridSearchCV(MLPClassifier(), param_grid, cv =5)

clf.fit(XTrain , yTrain)


GridSearchCV(cv=5, estimator=MLPClassifier(),
             param_grid={'activation': ['identity', 'logistic', 'tanh', 'relu'],
                         'hidden_layer_sizes': [3, 5],
                         'max_fun': [300, 500, 1000, 5000, 10000, 15000, 20000],
                         'max_iter': [200, 500, 1000], 'random_state': [125],
                         'solver': ['lbfgs']})

In [226]:
print("Mejor estimación de parámetros según GridSearchCV:")
print(clf.best_estimator_)
model = clf.best_estimator_

Mejor estimación de parámetros según GridSearchCV:
MLPClassifier(activation='tanh', hidden_layer_sizes=5, max_fun=300,
              random_state=125, solver='lbfgs')


# Cross Validation

In [227]:
print("Mejor resultado de la cross validation del modelo con mejores resultados: " +str(clf.best_score_))

Mejor resultado de la cross validation del modelo con mejores resultados: 0.6


# Predicciones y estudio de resultados

In [228]:
# Realizamos las predicciones con el modelo óptimo sobre el conjunto de datos de entrenamiento
yhatTrain = model.predict(XTrain)
contTrain = 0

# Comparamos con la columna Target y comprobamos cuantos aciertos ha habido
for i in range(0,len(yTrain),1) :
    if (yhatTrain[i] == yTrain[i]):
        contTrain = contTrain + 1

In [229]:
# Realizamos las predicciones con el modelo óptimo sobre el conjunto de datos de prueba
yhatTest = model.predict(XTest)
contTest = 0

# Comparamos con la columna Target y comprobamos cuantos aciertos ha habido
for i in range(0,len(yTest),1) :
    if (yhatTest[i] == yTest[i]):
        contTest = contTest + 1

In [230]:
print('Precisión final en el conjunto de datos de entrenamiento: ' + str(contTrain/len(yTrain)))
print('Precisión final en el conjunto de datos de prueba: ' + str(contTest/len(yTest)))

Precisión final en el conjunto de datos de entrenamiento: 0.9777777777777777
Precisión final en el conjunto de datos de prueba: 0.75


# Matriz de confusión
### 1. Matriz de confusión del conjunto de datos del entrenamiento

In [231]:
from sklearn.metrics import classification_report,confusion_matrix

print('----------------Matriz de confusión (Entrenamiento)------------------')
print(confusion_matrix(yTrain,yhatTrain))
print('Datos de entrada:  ' + str(np.array(yTrain)))
print('Predicción:        ' +str(yhatTrain))

----------------Matriz de confusión (Entrenamiento)------------------
[[22  1]
 [ 0 22]]
Datos de entrada:  [0 1 1 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 1 0 1 1 0 0 0 1 1 0 0 1 1 1 1 1 0 0
 0 1 0 1 0 1 1 1]
Predicción:        [0 1 1 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 1 0 1 1 0 0 0 1 1 1 0 1 1 1 1 1 0 0
 0 1 0 1 0 1 1 1]


Estos resultados pueden ser interpretados de la siguiente manera:
1. 22 verdaderos positivos (Verdaderamente responderán al tratamiento)
2. 22 verdaderos negativos (Verdaderamente no responderán al tratamiento)
3. 1 falsos positivos (Fueron clasificados como mala respuesta al tratamiento, sin embargo, responderán correctamente al mismo)
4. 0 falsos negativos (Fueron clasificados como buena respuesta al tratamiento, sin embargo, no responderán al tratamiento)

### 2. Resultados obtenidos del conjunto de entrenamiento

In [232]:
print(classification_report(yTrain,yhatTrain))

              precision    recall  f1-score   support

           0       1.00      0.96      0.98        23
           1       0.96      1.00      0.98        22

    accuracy                           0.98        45
   macro avg       0.98      0.98      0.98        45
weighted avg       0.98      0.98      0.98        45



### 3. Matriz de confusión del conjunto de datos de prueba

In [233]:
print('----------------Matriz de confusión (Prueba)------------------')
print(confusion_matrix(yTest,yhatTest))
print('Datos de entrada:  ' + str(np.array(yTest)))
print('Predicción:        ' +str(yhatTest))

----------------Matriz de confusión (Prueba)------------------
[[5 2]
 [0 1]]
Datos de entrada:  [0 0 0 1 0 0 0 0]
Predicción:        [1 0 0 1 0 0 0 1]


Estos resultados pueden ser interpretados de la siguiente manera:
1. 5 verdaderos positivos (Verdaderamente responderán al tratamiento)
2. 1 verdaderos negativos (Verdaderamente no responderán al tratamiento)
3. 2 falsos positivos (Fueron clasificados como mala respuesta al tratamiento, sin embargo, responderán correctamente al mismo)
4. 0 falsos negativos (Fueron clasificados como buena respuesta al tratamiento, sin embargo, no responderán al tratamiento)

### 4. Resultados obtenidos del conjunto de prueba

In [234]:
print(classification_report(yTest,yhatTest))

              precision    recall  f1-score   support

           0       1.00      0.71      0.83         7
           1       0.33      1.00      0.50         1

    accuracy                           0.75         8
   macro avg       0.67      0.86      0.67         8
weighted avg       0.92      0.75      0.79         8



# Bibliografía

https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html?highlight=mlpclassifier#sklearn.neural_network.MLPClassifier

https://moodle.upm.es/titulaciones/oficiales/pluginfile.php/10073620/mod_resource/content/1/Unit2-ClassificationRegression-NeuralNetworks.pdf

https://www.llipe.com/2017/04/19/programando-un-clasificador-perceptron-en-python/

https://machinelearningmastery.com/implement-perceptron-algorithm-scratch-python/